In [1]:
import os
from os import listdir
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import skimage.metrics
import cv2

In [ ]:
def cr(original_img, compressed_img):
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    compressed_img = cv2.cvtColor(compressed_img, cv2.COLOR_BGR2RGB)

    original_img=cv2.imwrite("original.png", original_img)
    
    compressed_img=cv2.imwrite("compressed.png", compressed_img)

    return round(os.path.getsize("original.png")/os.path.getsize("compressed.png"), 3)

ssim_vals = []
psnr_vals = []
cr_vals = []

# Number of components to use for PCA
pca_comp = 50
# Source folder for PCA
folder_dir = "./imagenet"

images_gray = []

homes_ind = -1
count = 0

for img in os.listdir(folder_dir):
    if (img.endswith(".jpg")):
        image = plt.imread(folder_dir + img)
        if (img.endswith("(1).jpg")):
            homes_ind = count

        if ((image.shape[0] != 2160) or (image.shape[1] != 3840)):
            continue
        image = np.reshape(image, (image.shape[0]*image.shape[1], -1))

        images_gray.append(image)
        count += 1

print(len(images_gray))
images_gray = np.stack(images_gray, axis = 0)
#images_gray = images_gray[:20, :, :]
images_gray_copy = np.copy(images_gray)

images_red = images_gray[:,:,0]
images_green = images_gray[:,:,1]
images_blue = images_gray[:,:,2]

print("reached here")
pca_red = PCA(n_components=pca_comp)
img_reduced_red = pca_red.fit_transform(images_red)
img_recovered_red = pca_red.inverse_transform(img_reduced_red)

print("done")

pca_green = PCA(n_components=pca_comp)
img_reduced_green = pca_green.fit_transform(images_green)
img_recovered_green = pca_green.inverse_transform(img_reduced_green)

pca_blue = PCA(n_components=pca_comp)
img_reduced_blue = pca_blue.fit_transform(images_blue)
img_recovered_blue = pca_blue.inverse_transform(img_reduced_blue)

recovered_imgs = np.stack([img_recovered_red, img_recovered_green, img_recovered_blue], axis = 2)

psnr = skimage.metrics.peak_signal_noise_ratio(images_gray_copy[homes_ind,:, :].reshape([2160, 3840, 3]), recovered_imgs[homes_ind, :, :].reshape([2160, 3840, 3]))
psnr_vals.append(psnr)
ssim = skimage.metrics.structural_similarity(images_gray_copy[homes_ind,:, :].reshape([2160, 3840, 3]), recovered_imgs[homes_ind, :, :].reshape([2160, 3840, 3]), channel_axis = 2)
ssim_vals.append(ssim)
comp_rat = cr(images_gray_copy[homes_ind,:, :].reshape([2160, 3840, 3]), recovered_imgs[homes_ind, :, :].reshape([2160, 3840, 3]))
cr_vals.append(comp_rat)

print(psnr_vals)
print(ssim_vals)
print(cr_vals)